In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleanee1/LabelEncoded.csv
/kaggle/input/cleanee/IQR_data.csv


In [2]:
dftrain = pd.read_csv('/kaggle/input/cleanee1/LabelEncoded.csv')

In [3]:
cat_cols = ['Street','City','County','State','Airport_Code','Wind_Direction','Weather_Condition','Sunrise_Sunset']

# Convert categorical columns to category type
for col in cat_cols:
    dftrain[col] = dftrain[col].astype('category')

In [4]:
for cols in dftrain.columns:
    print(cols, len(dftrain[cols].unique()))

Unnamed: 0 243019
Year 3
Severity 4
Start_Lat 134988
Start_Lng 134347
Distance(mi) 3674
Street 42361
City 6857
County 1277
State 49
Airport_Code 1616
Temperature(F) 271
Wind_Chill(F) 573
Visibility(mi) 49
Wind_Direction 23
Weather_Condition 70
Traffic_Signal 2
Sunrise_Sunset 2
TimeDiff 3822


In [5]:
df1 = dftrain.drop(columns=['Unnamed: 0'])

In [6]:
df_ = df1.drop(columns=['Severity'])

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import re as re
from collections import Counter

from tqdm.auto import tqdm
import math
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
import warnings
warnings.filterwarnings('ignore')

import time
from xgboost import XGBClassifier
%matplotlib inline
tqdm.pandas()


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
X = df1.drop(columns=['Severity'])
y = df1['Severity']
y = y-np.ones(y.shape[0])

In [9]:
y,df1['Severity']

(0         1.0
 1         2.0
 2         1.0
 3         2.0
 4         1.0
          ... 
 243014    1.0
 243015    1.0
 243016    3.0
 243017    3.0
 243018    3.0
 Name: Severity, Length: 243019, dtype: float64,
 0         2
 1         3
 2         2
 3         3
 4         2
          ..
 243014    2
 243015    2
 243016    4
 243017    4
 243018    4
 Name: Severity, Length: 243019, dtype: int64)

In [10]:
from sklearn.model_selection import train_test_split
X_train_, X_test, y_train_, y_test = train_test_split(X,y , random_state=42,test_size=0.20, shuffle=False)

In [11]:
FOLDS = 20
SEED = 1004
xgb_models = []
xgb_oof = []
predictions = np.zeros(((y_test.shape[0]),4))
f_imp = []
xgb_valAccu=[]
xgb_TrainAccu=[]

counter = 0
X = X_train_
y = y_train_
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    if (fold + 1) % 5 == 0 or (fold + 1) == 1:
        print(f'{"#" * 24} Training FOLD {fold + 1} {"#" * 24}')
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]
    watchlist = [(X_train, y_train), (X_valid, y_valid)]

    # XGboost model and fit with GPU support
    model = XGBClassifier(n_estimators=250, n_jobs=-1, max_depth=3, eta=0.2, colsample_bytree=0.67, tree_method='gpu_hist',enable_categorical=True,verbosity=0)
    model.fit(X_train, y_train, eval_set=watchlist, early_stopping_rounds=300,verbose=False)

    val_preds = model.predict_proba(X_valid)
    train_preds = model.predict_proba(X_train)
    val_pred = model.predict(X_valid)
#     val_accu = accuracy_score(y_valid, val_preds)
#     print(val_accu)
    val_score = roc_auc_score(y_valid, val_preds,multi_class='ovr')
    train_score = roc_auc_score(y_train, train_preds,multi_class='ovr')
    best_iter = model.best_iteration
    xgb_valAccu.append(val_score)
    xgb_TrainAccu.append(train_score)
#     idx_pred_target = np.vstack([val_idx, val_preds, y_valid]).T  # shape(len(val_idx), 3)
#     f_imp.append({i: j for i, j in zip(X.columns, model.feature_importances_)})
    print(f'{" " * 20}AUC: {val_score:.5f} {" " * 6}Best Iteration: {best_iter}')

    if val_score > 0.80 and train_score > 0.90:
        test_preds = model.predict_proba(X_test)
        predictions += test_preds
        counter += 1

if(counter!=0):
    predictions /= counter
# mean_val_auc = get_mean_auc(np.array(xgb_oof))


######################## Training FOLD 1 ########################
                    AUC: 0.92216       Best Iteration: 249
                    AUC: 0.84896       Best Iteration: 249
                    AUC: 0.90635       Best Iteration: 249
                    AUC: 0.93410       Best Iteration: 249
######################## Training FOLD 5 ########################
                    AUC: 0.78257       Best Iteration: 249
                    AUC: 0.78250       Best Iteration: 249
                    AUC: 0.89778       Best Iteration: 249
                    AUC: 0.85127       Best Iteration: 249
                    AUC: 0.87463       Best Iteration: 249
######################## Training FOLD 10 ########################
                    AUC: 0.85724       Best Iteration: 249
                    AUC: 0.82007       Best Iteration: 249
                    AUC: 0.93728       Best Iteration: 249
                    AUC: 0.88136       Best Iteration: 249
                    AUC: 0.90397  

In [12]:
y_test.shape

(48604,)

In [13]:
predictions.shape

(48604, 4)

In [14]:
print("Testing score")
# print(roc_auc_score(y_test, predictions,multi_class='ovr',labels=[0,1,2,3]))

Testing score


In [15]:
XGBpredictions = []
for i in predictions:
    XGBpredictions.append(np.argmax(i))
print("XGB test confusion matrix")
confusion_matrix(y_test,XGBpredictions,labels=[0,1,2,3])    

XGB test confusion matrix


array([[    0,     5,     0,     0],
       [    1, 20245, 10634,     1],
       [    0,  3984,  9172,     1],
       [    0,  2942,  1618,     1]])

In [16]:
print("Mean of Training score")
print(np.mean(xgb_TrainAccu))


Mean of Training score
0.9875262827359712


In [17]:
print("Mean of Validation score")
print(np.mean(xgb_valAccu))


Mean of Validation score
0.8696126840673214


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from scipy.stats import median_abs_deviation
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [19]:
sc = StandardScaler()
X_train_  = sc.fit_transform(X_train_)
X_test = sc.transform(X_test)

In [20]:
rf = RandomForestClassifier(criterion = "entropy")
rf = rf.fit(X_train_, y_train_)

y_pred = rf.predict(X_test)
print("Random forest accuracy: ", accuracy_score(y_test,y_pred)*100)

print(classification_report(y_test, y_pred))

Random forest accuracy:  61.18220722574274
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         5
         1.0       0.74      0.67      0.70     30881
         2.0       0.44      0.69      0.54     13157
         3.0       0.00      0.00      0.00      4561

    accuracy                           0.61     48604
   macro avg       0.29      0.34      0.31     48604
weighted avg       0.59      0.61      0.59     48604



In [21]:
print("Random Forest")
print(confusion_matrix(y_test, y_pred,labels=[0,1,2,3]))

Random Forest
[[    0     5     0     0]
 [    0 20640 10241     0]
 [    0  4060  9097     0]
 [    0  3055  1506     0]]


In [22]:
rfpredictions = rf.predict_proba(X_test)
ensemblepredictions = rfpredictions+predictions
Ensemblepredictions = []
for i in ensemblepredictions:
    Ensemblepredictions.append(np.argmax(i))
print("Ensemble confusion matrix")
confusion_matrix(y_test,Ensemblepredictions,labels=[0,1,2,3]) 

Ensemble confusion matrix


array([[    0,     5,     0,     0],
       [    0, 20516, 10365,     0],
       [    0,  3934,  9223,     0],
       [    0,  3027,  1534,     0]])